### 什么是文本张量表示
将一段文本使用张量进行表示，其中一般将词汇为表示成向量，称作词向量，再由各个词向量按顺序组成矩阵形成文本表示.

In [4]:
# 举个例子
# ["人生", "该", "如何", "起头"]
# 每个词对应矩阵中的一个向量
# [[1.32, 4,32, 0,32, 5.2],
#  [3.1, 5.43, 0.34, 3.2],
#  [3.21, 5.32, 2, 4.32],
#  [2.54, 7.32, 5.12, 9.54]]

### 文本张量表示的作用
将文本表示成张量（矩阵）形式，能够使语言文本可以作为计算机处理程序的输入，进行接下来一系列的解析工作.

### 文本张量表示的方法:
- one-hot编码
- Word2vec
- Word Embedding

### 什么是one-hot词向量表示
又称独热编码，将每个词表示成具有n个元素的向量，这个词向量中只有一个元素是1，其他元素都是0，不同词汇元素为0的位置不同，其中n的大小是整个语料中不同词汇的总数

["改变", "要", "如何", "起手"]`
==>

[[1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1]]

### onehot编码实现:

In [5]:
# 导入用于对象保存与加载的joblib
import joblib
# 导入keras中的词汇映射器Tokenizer
from keras.preprocessing.text import Tokenizer

# 假定vocab为语料集所有不同词汇集合
vocab = {"周杰伦", "陈奕迅", "王力宏", "李宗盛", "鹿晗", "吴亦凡"}
# 实例化一个词汇映射器对象
t = Tokenizer(num_words=None, char_level=False)
# 使用映射器拟合现有文本数据
t.fit_on_texts(vocab)

for token in vocab:
    zero_list = [0] * len(vocab)
    # 使用映射器转化现有文本数据, 每个词汇对应从1开始的自然数
    # 返回样式如: [[2]], 取出其中的数字需要使用[0][0]
    token_index = t.texts_to_sequences([token])[0][0] - 1
    zero_list[token_index] = 1
    print(token, "的one-hot编码为:", zero_list)

# 使用joblib工具保存映射器, 以便之后使用
tokenizer_path = "./Tokenizer"
joblib.dump(t, tokenizer_path)

李宗盛 的one-hot编码为: [1, 0, 0, 0, 0, 0]
王力宏 的one-hot编码为: [0, 1, 0, 0, 0, 0]
陈奕迅 的one-hot编码为: [0, 0, 1, 0, 0, 0]
鹿晗 的one-hot编码为: [0, 0, 0, 1, 0, 0]
周杰伦 的one-hot编码为: [0, 0, 0, 0, 1, 0]
吴亦凡 的one-hot编码为: [0, 0, 0, 0, 0, 1]


['./Tokenizer']

In [6]:
# 导入用于对象保存与加载的joblib
# from sklearn.externals import joblib
# 加载之前保存的Tokenizer, 实例化一个t对象
t = joblib.load(tokenizer_path)

# 编码token为"李宗盛"
token = "李宗盛"
# 使用t获得token_index
token_index = t.texts_to_sequences([token])[0][0] - 1
# 初始化一个zero_list
zero_list = [0] * len(vocab)
# 令zero_List的对应索引为1
zero_list[token_index] = 1
print(token, "的one-hot编码为:", zero_list)

李宗盛 的one-hot编码为: [1, 0, 0, 0, 0, 0]


### one-hot编码的优劣势：
- 优势：操作简单，容易理解.
- 劣势：完全割裂了词与词之间的联系，而且在大语料集下，每个向量的长度过大，占据大量内存

### 什么是word2vec
是一种流行的将词汇表示成向量的无监督训练方法, 该过程将构建神经网络模型, 将网络参数作为词汇的向量表示, 它包含CBOW和skipgram两种训练模式.
- CBOW给定一段用于训练的文本语料, 再选定某段长度(窗口)作为研究对象, 使用上下文词汇预测目标词汇.
- skipgram给定一段用于训练的文本语料, 再选定某段长度(窗口)作为研究对象, 使用目标词汇预测上下文词汇.

### 使用fasttext工具实现word2vec的训练和使用
- 第一步: 获取训练数据
- 第二步: 训练词向量
- 第三步: 模型超参数设定
- 第四步: 模型效果检验
- 第五步: 模型的保存与重加载

### 第一步: 获取训练数据
#在这里, 我们将研究英语维基百科的部分网页信息, 它的大小在300M左右
#这些语料已经被准备好, 我们可以通过Matt Mahoney的网站下载.
#首先创建一个存储数据的文件夹data
#mkdir data
#使用wget下载数据的zip压缩包, 它将存储在data目录中
#wget -c http://mattmahoney.net/dc/enwik9.zip -P data
#使用unzip解压, 如果你的服务器中还没有unzip命令, 请使用: yum install unzip -y
#解压后在data目录下会出现enwik9的文件夹
#unzip data/enwik9.zip -d data
#使用wikifil.pl文件处理脚本来清除XML/HTML格式的内容
#注: wikifil.pl文件已为大家提供
#$perl wikifil.pl data/enwik9 > data/fil9

### 第二步: 训练词向量

In [1]:
# 导入fasttext
import fasttext

# 使用fasttext的train_unsupervised(无监督训练方法)进行词向量的训练
# 它的参数是数据集的持久化文件路径'data/fil9'
# 这个是默认版本的模式
model = fasttext.train_unsupervised('data/fil9')

# 下面为终端打印信息
# 有效训练词汇量为124M, 共218316个单词
# Read 124M words
# Number of words:  218316
# Number of labels: 0
# Progress: 100.0% words/sec/thread:   53996 lr:  0.000000 loss:  0.734999 ETA:   0h 0m

In [2]:
# 查看单词对应的词向量:
# 通过get_word_vector方法来获得指定词汇的词向量
model.get_word_vector("the")

array([ 0.34561405,  0.14688048,  0.01107159,  0.03166007, -0.05890983,
        0.0245652 , -0.09022985,  0.0045212 , -0.01121075,  0.17789716,
        0.3121547 ,  0.07821867, -0.06836791, -0.08550898, -0.21353933,
       -0.08400505,  0.01348103, -0.27766767, -0.03285928, -0.31067395,
       -0.08447772, -0.21946745,  0.08424345, -0.04118999, -0.05621148,
       -0.0491033 ,  0.14507769, -0.13851337, -0.06879391,  0.32776594,
        0.07838137,  0.00473783,  0.07279453, -0.12495724,  0.01784861,
        0.28451672, -0.12206005,  0.08648696, -0.12739372,  0.19510774,
        0.04876711,  0.37033972,  0.35425773, -0.02962892, -0.02837615,
       -0.37998664,  0.22328205, -0.11799385, -0.09353255,  0.05917957,
        0.20065686, -0.28623486,  0.28301543, -0.09517141,  0.0697737 ,
       -0.15205644,  0.02752621, -0.10729717,  0.04979468,  0.17059511,
       -0.22716118,  0.2890576 , -0.02532258,  0.25126162,  0.1504471 ,
        0.05447647,  0.00519024, -0.04062964, -0.04412157, -0.01

###  第三步：模型超参数设定

In [4]:
# 在训练词向量过程中, 我们可以设定很多常用超参数来调节我们的模型效果, 如:
# 无监督训练模式: 'skipgram' 或者 'cbow', 默认为'skipgram', 在实践中，skipgram模式在利用子词方面比cbow更好.
# 词嵌入维度dim: 默认为100, 但随着语料库的增大, 词嵌入的维度往往也要更大.
# 数据循环次数epoch: 默认为5, 但当你的数据集足够大, 可能不需要那么多次.
# 学习率lr: 默认为0.05, 根据经验, 建议选择[0.01，1]范围内.
# 使用的线程数thread: 默认为12个线程, 一般建议和你的cpu核数相同.

#model = fasttext.train_unsupervised('data/fil9', "cbow", dim=300, epoch=1, lr=0.1, thread=8)

# Read 124M words
# Number of words:  218316
# Number of labels: 0
# Progress: 100.0% words/sec/thread:   49523 lr:  0.000000 avg.loss:  1.777205 ETA:   0h 0m 0s

### 第四步: 模型效果检验

In [5]:
# 检查单词向量质量的一种简单方法就是查看其邻近单词, 通过我们主观来判断这些邻近单词是否与目标单词相关来粗略评定模型效果好坏.

# 查找"音乐"的邻近单词, 我们可以发现与音乐有关的词汇.
model.get_nearest_neighbors('music')

[(0.8949843645095825, 'musics'),
 (0.8442449569702148, 'musical'),
 (0.8227779865264893, 'musicman'),
 (0.8069064021110535, 'musices'),
 (0.797702968120575, 'musico'),
 (0.7893351316452026, 'emusic'),
 (0.7851594090461731, 'afrobeat'),
 (0.7760413289070129, 'jazz'),
 (0.7743896842002869, 'musicweb'),
 (0.7724300026893616, 'folksongs')]

### 第五步: 模型的保存与重加载

In [6]:
# 使用save_model保存模型
model.save_model("data/fil9.bin")

# 使用fasttext.load_model加载模型
model = fasttext.load_model("data/fil9.bin")
model.get_word_vector("the")

array([ 0.34561405,  0.14688048,  0.01107159,  0.03166007, -0.05890983,
        0.0245652 , -0.09022985,  0.0045212 , -0.01121075,  0.17789716,
        0.3121547 ,  0.07821867, -0.06836791, -0.08550898, -0.21353933,
       -0.08400505,  0.01348103, -0.27766767, -0.03285928, -0.31067395,
       -0.08447772, -0.21946745,  0.08424345, -0.04118999, -0.05621148,
       -0.0491033 ,  0.14507769, -0.13851337, -0.06879391,  0.32776594,
        0.07838137,  0.00473783,  0.07279453, -0.12495724,  0.01784861,
        0.28451672, -0.12206005,  0.08648696, -0.12739372,  0.19510774,
        0.04876711,  0.37033972,  0.35425773, -0.02962892, -0.02837615,
       -0.37998664,  0.22328205, -0.11799385, -0.09353255,  0.05917957,
        0.20065686, -0.28623486,  0.28301543, -0.09517141,  0.0697737 ,
       -0.15205644,  0.02752621, -0.10729717,  0.04979468,  0.17059511,
       -0.22716118,  0.2890576 , -0.02532258,  0.25126162,  0.1504471 ,
        0.05447647,  0.00519024, -0.04062964, -0.04412157, -0.01

### 什么是word embedding(词嵌入)
- 通过一定的方式将词汇映射到指定维度(一般是更高维度)的空间.
- 广义的word embedding包括所有密集词汇向量的表示方法，如之前学习的word2vec, 即可认为是word embedding的一种.
- 狭义的word embedding是指在神经网络中加入的embedding层, 对整个网络进行训练的同时产生的embedding矩阵(embedding层的参数), 这个embedding矩阵就是训练过程中所有输入词汇的向量表示组成的矩阵.

In [27]:
# word embedding的可视化分析:
# 导入torch和tensorboard的摘要写入方法
import torch
import json
import fileinput
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
import tensorboard as tb

tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

# 实例化一个摘要写入对象
writer = SummaryWriter()

# 随机初始化一个100x50的矩阵, 认为它是我们已经得到的词嵌入矩阵
# 代表100个词汇, 每个词汇被表示成50维的向量
embedded = torch.randn(100, 50)

# 导入事先准备好的100个中文词汇文件, 形成meta列表原始词汇
meta = list(map(lambda x: x.strip(), fileinput.FileInput("vocab100.csv", openhook=fileinput.hook_encoded("utf-8", ''))))
writer.add_embedding(embedded, metadata=meta)
writer.close()

In [ ]:
# 在终端启动tensorboard服务:
!tensorboard --logdir runs --host 0.0.0.0